# K-Means Clustering

Import dependencies

In [1]:
import glob
import cv2
import numpy as np

from sklearn.cluster import KMeans
from skimage.color import rgb2gray

Load images

In [2]:
images = glob.glob("./*/*/*/[0-9]*[0-9].png")

Train K-Means algorithm on example image of each sensor

In [3]:
def train_kmeans(image_path, cluster_size):
    #load image data
    image = cv2.imread(image_path)
    #Reshape data to 2D array
    image_data = np.reshape(image, (image.shape[0] * image.shape[1], 3))
    #Train KMeans with given parameters
    kmeans = KMeans(n_clusters = cluster_size, random_state=0).fit(image_data)

    return kmeans

In [4]:
def apply_kmeans(image_path, models, sensor, cluster_size):
    #Load image data
    image = cv2.imread(image_path)
    #Reshape data to 2D array
    image_data = np.reshape(image, (image.shape[0] * image.shape[1], 3))
    #Apply model to image data
    prediction = models["{}_{}".format(sensor, str(cluster_size))].predict(image_data)
    #Reshape result to image dimensions
    prediction  = np.reshape(prediction, (image.shape[0], image.shape[1], 1))

    return prediction

In [5]:
cluster_sizes = range(2, 7)

In [6]:
FX10_train_image = glob.glob("./*/*/*/20220124_FX10__ras5_1-30_A_2022-01-24_09-52-45.png")[0]
FX17_train_image = glob.glob("./*/*/*/20220124_FX17__ras5_1-30_A_2022-01-24_09-52-46.png")[0]

models = {}

for idx in cluster_sizes:
    models["FX10_{}".format(idx)] = train_kmeans(FX10_train_image, idx)
    models["FX17_{}".format(idx)] = train_kmeans(FX17_train_image, idx)

models

c:\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` w

{'FX10_2': KMeans(n_clusters=2, random_state=0),
 'FX17_2': KMeans(n_clusters=2, random_state=0),
 'FX10_3': KMeans(n_clusters=3, random_state=0),
 'FX17_3': KMeans(n_clusters=3, random_state=0),
 'FX10_4': KMeans(n_clusters=4, random_state=0),
 'FX17_4': KMeans(n_clusters=4, random_state=0),
 'FX10_5': KMeans(n_clusters=5, random_state=0),
 'FX17_5': KMeans(n_clusters=5, random_state=0),
 'FX10_6': KMeans(n_clusters=6, random_state=0),
 'FX17_6': KMeans(n_clusters=6, random_state=0)}

In [7]:
FX10_test_image = images[0]  #FX10 R1 1-15 A 21
FX17_test_image = images[73] #FX17 R1 1-15 A 21

print(FX10_test_image, FX17_test_image)

for model in models:
    details = model.split("_")
    prediction = apply_kmeans(FX10_test_image if "FX10" in model else FX17_test_image, models, details[0], details[1])

    if model == "FX10_2":
        prediction = prediction == 1
    elif model == "FX17_2":
        prediction = prediction == 0
    elif model == "FX10_3":
        prediction = prediction == 0
    elif model == "FX17_3":
        prediction = prediction == 2
    elif model == "FX10_4":
        prediction = prediction == 3
    elif model == "FX17_4":
        fg_cluster_0 = prediction == 0
        fg_cluster_1 = prediction == 3
        prediction =  fg_cluster_0 | fg_cluster_1
    elif model == "FX10_5":
        fg_cluster_0 = prediction == 2
        fg_cluster_1 = prediction == 3
        prediction =  fg_cluster_0 | fg_cluster_1
    elif model == "FX17_5":
        fg_cluster_0 = prediction == 1
        fg_cluster_1 = prediction == 2
        fg_cluster_2 = prediction == 4
        prediction =  fg_cluster_0 | fg_cluster_1 | fg_cluster_2
    elif model == "FX10_6":
        fg_cluster_0 = prediction == 0
        fg_cluster_1 = prediction == 3
        prediction =  fg_cluster_0 | fg_cluster_1
    elif model == "FX17_6":
        fg_cluster_0 = prediction == 2
        fg_cluster_1 = prediction == 4
        fg_cluster_2 = prediction == 5
        prediction =  fg_cluster_0 | fg_cluster_1 | fg_cluster_2
    
    cv2.imwrite("./KMeans_cluster_test/{}_test.png".format(model), prediction.astype(np.uint8)*255)

.\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\20220121_FX10__ras1_1-15_2022-01-21_09-12-35.png .\Analysis_BP\FX17\20220121_FX17__ras1_1-15_A_2022-01-21_09-12-35\20220121_FX17__ras1_1-15_2022-01-21_09-12-35.png


In [85]:
def evaluate_image(image, save_name, wref_path, control = "", wref = False):
    image_path = image.replace(image.split("\\")[-1], "")

    control = cv2.imread("{}{}.png".format(image_path, control)).reshape(-1) if control != "" else cv2.imread("{}labelbox_mask_combination.png".format(image_path)).reshape(-1)
    
    to_be_evaluated = cv2.imread("{}{}".format(image_path, save_name))


    shape = to_be_evaluated.shape
    result = np.full(shape, (0, 0, 0), dtype=np.uint8)

    if wref:
        to_be_evaluated_wref = cv2.imread(wref_path)

        result = to_be_evaluated & to_be_evaluated_wref

        to_be_evaluated = result.reshape(-1)
    else:
        to_be_evaluated.reshape(-1)
    

    stats = {
    "total": control.shape[0],
    "control_fg" : 0,
    "control_bg" : 0,
    "eval_fg" : 0,
    "eval_bg" : 0,
    "correct" : 0,
    "correct_bg" : 0,
    "correct_fg" : 0,
    "false" : 0,
    "false_bg" : 0,
    "false_fg" : 0,
    }

    for c, e in zip(control, to_be_evaluated):
        if c == 0:
            stats["control_bg"] += 1
        else:
            stats["control_fg"] += 1

        if e == 0:
            stats["eval_bg"] += 1
        else:
            stats["eval_fg"] += 1

        if c == e:
            stats["correct"] += 1
            if c == 0:
                stats["correct_bg"] += 1
            else:
                stats["correct_fg"] += 1 
        if c != e:
            stats["false"] += 1
            if c == 0:
                stats["false_bg"] += 1
            else:
                stats["false_fg"] += 1 

    file = open("{}results_evaluation_{}.txt".format(image_path, save_name.split(".")[0]), "w")
    file.write("{}".format(image.split("\\")[-1]).split(".png")[0] + "\n\n")

    for stat in stats:
        file.write("{}: {}\n".format(stat, stats[stat]))
        
    file.write("\n")
    for stat in stats:
        file.write("{} (%): {}\n".format(stat, stats[stat] * 100 / control.shape[0]))

    file.close()

    print("Evaluated: {}{}".format(image_path, save_name.split(".")[0]))

    return to_be_evaluated, shape

In [86]:
for model in models:
    eval_img, shape = evaluate_image(".\\KMeans_cluster_test\\{}_test.png".format(model), "{}_test.png".format(model), 
    wref_path = ".\\KMeans_cluster_test\\FX10_wref_mask.png" if "FX10" in model else ".\\KMeans_cluster_test\\FX17_wref_mask.png",
    control = "FX10_labelbox" if "FX10" in model else "FX17_labelbox", wref = True)

    eval_img = eval_img.reshape(shape)
    
    cv2.imwrite("./KMeans_cluster_test/{}_test_wref.png".format(model), eval_img)
    # print(shape)

Evaluated: .\KMeans_cluster_test\FX10_2_test
Evaluated: .\KMeans_cluster_test\FX17_2_test
Evaluated: .\KMeans_cluster_test\FX10_3_test
Evaluated: .\KMeans_cluster_test\FX17_3_test
Evaluated: .\KMeans_cluster_test\FX10_4_test
Evaluated: .\KMeans_cluster_test\FX17_4_test
Evaluated: .\KMeans_cluster_test\FX10_5_test
Evaluated: .\KMeans_cluster_test\FX17_5_test
Evaluated: .\KMeans_cluster_test\FX10_6_test
Evaluated: .\KMeans_cluster_test\FX17_6_test


Apply K-Means Clustering algorithm to each image an save the mask

In [8]:
save_name = "k-means_{}_clusters.png"

for image in images:
    #Construct save path
    save_path = image.replace(image.split("\\")[-1], "")

    #Load image to memory in grayscale
    cached_image = cv2.imread(image)

    # #Detect sensor and set threshold value accordingly
    sensor = "FX10" if "FX10" in image else "FX17"

    # #Apply k-means to the image
    result = apply_kmeans(image, models, sensor, 4)

    if sensor == "FX10":
        result = result == 3
    elif sensor == "FX17":
        fg_cluster_0 = result == 0
        fg_cluster_1 = result == 3
        result =  fg_cluster_0 | fg_cluster_1

    #Save the binary image
    cv2.imwrite('{}{}'.format(save_path, save_name.format("4")), result.astype(np.uint8)*255)

Evaluate the mask quality and save the results

In [34]:
for image in images:
    image_path = image.replace(image.split("\\")[-1], "")
    control = cv2.imread("{}labelbox_mask_combination.png".format(image_path)).reshape(-1)
    to_be_evaluated = cv2.imread("{}{}".format(image_path, save_name)).reshape(-1)

    stats = {
    "total": control.shape[0],
    "correct" : 0,
    "correct_bg" : 0,
    "correct_fg" : 0,
    "false" : 0,
    "false_bg" : 0,
    "false_fg" : 0,
    }

    for c, e in zip(control, to_be_evaluated):
        if c == e:
            stats["correct"] += 1
            if c == 0:
                stats["correct_bg"] += 1
            else:
                stats["correct_fg"] += 1 
        if c != e:
            stats["false"] += 1
            if c == 0:
                stats["false_bg"] += 1
            else:
                stats["false_fg"] += 1 

    file = open("{}results_evaluation_kmeans.txt".format(image_path), "w")
    file.write("{}\n".format(image.split("\\")[-1]).split(".png")[0])
    file.write("\n")

    for stat in stats:
        file.write("{}: {}\n".format(stat, stats[stat]))
        
    file.write("\n")
    for stat in stats:
        file.write("{} (%): {}\n".format(stat, stats[stat] * 100 / control.shape[0]))

    file.close()